**Use PyTorch (by switching to a different kernel) to build a simple fully-connected artificial neural network for the beans classification based on the chosen features provided in the data. Generate a confusion matrix for the test data set to demonstrate the accuracy of the model. Based on your model, classify the beans provided in the unlabeled beans-unknown.csv data set. Indicate which classification has been assigned to each of the unlabeled beans. How do the results with the artificial neural network compare to the support vector machine model?**

In [62]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn 
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [36]:
## Loading the dataset
bean = pd.read_csv('/public/bmort/python/beans.csv')
bean = bean[['Area', 'AspectRatio','Extent','Solidity', 'roundness','ShapeFactor4','Class']]
bean.head()

,Area,AspectRatio,Extent,Solidity,roundness,ShapeFactor4,Class
0,28395,1.197191,0.763923,0.988856,0.958027,0.998724,SEKER
1,28734,1.097356,0.783968,0.984986,0.887034,0.998430,SEKER
2,29380,1.209713,0.778113,0.989559,0.947849,0.999066,SEKER
3,30008,1.153638,0.782681,0.976696,0.903936,0.994199,SEKER
4,30140,1.060798,0.773098,0.990893,0.984877,0.999166,SEKER


In [37]:
## Using the MinMaxScaler model to tranform the the selected columns
# create a scaler object
features_names = bean[['Area', 'AspectRatio', 'Extent', 'Solidity','roundness','ShapeFactor4']]
scaler = MinMaxScaler()

# fit and transform the data
scaled = pd.DataFrame(scaler.fit_transform(features_names), 
                       columns=features_names.columns)

scaled.head()

,Area,AspectRatio,Extent,Solidity,roundness,ShapeFactor4
0,0.034053,0.122612,0.671024,0.922824,0.934823,0.980620
1,0.035500,0.051577,0.735504,0.871514,0.793138,0.974979
2,0.038259,0.131521,0.716671,0.932141,0.914511,0.987196
3,0.040940,0.091623,0.731365,0.761614,0.826871,0.893675
4,0.041504,0.025565,0.700538,0.949832,0.988408,0.989116


In [38]:
## Replacing the transformed columns into the main data
bean[['Area', 'AspectRatio', 'Extent', 'Solidity','roundness','ShapeFactor4']] = scaled[['Area', 'AspectRatio', 'Extent', 
                                                                                'Solidity','roundness','ShapeFactor4']]
bean.head()

,Area,AspectRatio,Extent,Solidity,roundness,ShapeFactor4,Class
0,0.034053,0.122612,0.671024,0.922824,0.934823,0.980620,SEKER
1,0.035500,0.051577,0.735504,0.871514,0.793138,0.974979,SEKER
2,0.038259,0.131521,0.716671,0.932141,0.914511,0.987196,SEKER
3,0.040940,0.091623,0.731365,0.761614,0.826871,0.893675,SEKER
4,0.041504,0.025565,0.700538,0.949832,0.988408,0.989116,SEKER


In [39]:
## Replacing the class columns with labels
l_eco = LabelEncoder()
l_eco.fit(bean['Class'])
bean['Class'] = l_eco.transform(bean['Class'])

In [40]:
## Splitting the data into the predictors and target varible
X = bean[['Area', 'AspectRatio','Extent','Solidity','roundness','ShapeFactor4']].to_numpy()

In [41]:
X

array([[0.03405267, 0.12261211, 0.67102371, 0.92282385, 0.93482256,
        0.98061988],
       [0.03550018, 0.05157739, 0.73550396, 0.87151366, 0.79313798,
        0.97497943],
       [0.03825855, 0.13152124, 0.71667069, 0.9321406 , 0.9145106 ,
        0.98719586],
       ...,
       [0.09273856, 0.31855826, 0.56168866, 0.93664773, 0.85578518,
        0.9430251 ],
       [0.09277272, 0.33047232, 0.48274074, 0.90899135, 0.83479471,
        0.91334232],
       [0.09282396, 0.42333667, 0.75156921, 0.93332197, 0.7958257 ,
        0.9701623 ]])

In [42]:
y = bean['Class'].to_numpy()

In [43]:
## Train and test data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [44]:
# input layer nodes = 6 = number of features
# hidden layer nodes = 3 
# output layer nodes = 6 = number of categories
iln = 6
hln = 10
oln = 7
l_r = 0.01
num_epoch = 100000

In [45]:
## fully connected
class NNetwork(nn.Module):
    def __init__(self):
        super(NNetwork, self).__init__()
        self.fc1 = nn.Linear(iln, hln)
        self.out = nn.Linear(hln, oln)
    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.out(x)
        out = nn.functional.softmax(x, dim = 1)
        return out

In [46]:
## Setting up the classifer 
classifier = NNetwork()

In [47]:
## Defining loss function and optimizers
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classifier.parameters(), lr=l_r )

In [48]:
X_torch = torch.Tensor(X_train).float()
y_torch = torch.Tensor(y_train).long()
# X_torch
# y_torch.unique()

In [49]:
## Training our data
for epoch in range(num_epoch):
    optimizer.zero_grad()
    output = classifier(X_torch)
    loss = criterion(output, y_torch)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print('epoch:', epoch, 'loss:', loss.item())

epoch: 0 loss: 1.9462968111038208
epoch: 1000 loss: 1.925895094871521
epoch: 2000 loss: 1.8966991901397705
epoch: 3000 loss: 1.8864113092422485
epoch: 4000 loss: 1.883758544921875
epoch: 5000 loss: 1.881467580795288
epoch: 6000 loss: 1.8793209791183472
epoch: 7000 loss: 1.8774968385696411
epoch: 8000 loss: 1.8759421110153198
epoch: 9000 loss: 1.8744826316833496
epoch: 10000 loss: 1.8729615211486816
epoch: 11000 loss: 1.8712364435195923
epoch: 12000 loss: 1.8691270351409912
epoch: 13000 loss: 1.866313099861145
epoch: 14000 loss: 1.8620822429656982
epoch: 15000 loss: 1.8547999858856201
epoch: 16000 loss: 1.84213387966156
epoch: 17000 loss: 1.8248512744903564
epoch: 18000 loss: 1.807938814163208
epoch: 19000 loss: 1.7948945760726929
epoch: 20000 loss: 1.7857009172439575
epoch: 21000 loss: 1.7790424823760986
epoch: 22000 loss: 1.7736531496047974
epoch: 23000 loss: 1.7685465812683105
epoch: 24000 loss: 1.7633264064788818
epoch: 25000 loss: 1.7584114074707031
epoch: 26000 loss: 1.75401699542

In [50]:
X_ttest = torch.Tensor(X_test).float()
y_ttest = torch.Tensor(y_test).long()

In [51]:
f_output = classifier(X_ttest)

In [52]:
bean_new = pd.read_csv('/public/bmort/python/beans-unknown.csv')

In [53]:
(values, pred) = torch.max(f_output.data, dim = 1 )

In [54]:
pred

tensor([6, 2, 3,  ..., 3, 3, 4])

In [55]:
print(f'The Accuracy of the model is {(torch.sum(y_ttest==pred)/len(y_ttest))*100}%')

The Accuracy of the model is 62.68932342529297%


In [56]:
confusion_matrix(y_ttest, pred)

array([[  0,   0, 180,   1,  11,   0,  89],
       [  0,   0, 117,   0,   0,   0,   0],
       [  0,   0, 313,   2,   7,   0,  14],
       [  0,   0,   0, 609,   2,   0,  65],
       [  0,   0,  19,   1, 374,   0,   3],
       [  0,   0,   0, 375,   0,   0,  14],
       [  0,   0,   6,  99,   5,   0, 401]])

In [71]:
new_data = pd.read_csv('/public/bmort/python/beans-unknown.csv')[['Area', 'AspectRatio', 'Extent', 'Solidity','roundness','ShapeFactor4']]

In [72]:
new_data

,Area,AspectRatio,Extent,Solidity,roundness,ShapeFactor4
0,37500,1.586948,0.703406,0.988299,0.888690,0.995836
1,37500,1.549351,0.786229,0.992142,0.920295,0.998631
2,37511,1.490660,0.717365,0.990573,0.913474,0.998379
3,37513,1.518721,0.780545,0.987678,0.909270,0.998076
4,37514,1.521158,0.793309,0.989293,0.894773,0.997545


In [90]:
X_newd = torch.tensor(new_data.to_numpy()).float()

In [92]:
## Predicting the new dataset
classifier(X_newd)

tensor([[0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]], grad_fn=<SoftmaxBackward>)

## Fitting the SVM model

In [77]:
# Creating an SVM classifier with a linear kernel
svm_clf = svm.SVC(kernel='linear')

##training the model
svm_clf.fit(X_train,y_train)

SVC(kernel='linear')

In [78]:
svm_clf.predict(X_test)

array([6, 1, 3, ..., 3, 5, 4])

In [83]:
svm_clf.score(X,y)

0.9133968816965935

In [85]:
print(f'The accuracy for the SVM model is {round(svm_clf.score(X,y),4)*100}%')

The accuracy for the SVM model is 91.34%


In [80]:
## Predicting the new model
X_new = new_data.to_numpy()

In [86]:
## Prediciting the new dataset
svm_clf.predict(X_new)

array([1, 1, 1, 1, 1])

**Comparing the two models, we can see that the SVM model has a higher accuracy than the neural network model and therefore can guarantee us accurate classifications of objects into their actual groups.**